# Summary


----

# Imports

In [1]:
%run imports.ipynb

2016-08-25 21:37:26.777347


In [2]:
%run db.ipynb

MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-25 21:37:26.932492


In [16]:
NOTEBOOK_NAME = 'humsavar'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Export from remote db

In [60]:
DB_URL = kmtools.db_tools.parse_connection_string(db_remote.connection_string)['db_url']

In [7]:
# SQL query
sql_query = r"""\
SELECT
    uniprot_id uniprot_id,
    uniprot_mutation mutation,
    null ddg_exp,
    null del_score_exp,
    CASE type_of_variant WHEN 'Polymorphism' THEN 0 WHEN 'Disease' then 1 ELSE NULL END del_class_exp
INTO OUTFILE '/tmp/{}.tsv' 
    FIELDS TERMINATED BY '\t' OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
FROM uniprot.humsavar
WHERE type_of_variant = 'Polymorphism' OR type_of_variant = 'Disease';
""".format(NOTEBOOK_NAME)
print_sql(sql_query)

In [8]:
db_remote.engine.execute(sql_query)

# Load to local db

In [39]:
!rsync -av {DB_URL}:/tmp/{NOTEBOOK_NAME}.tsv {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv

receiving incremental file list
humsavar.tsv

sent 43 bytes  received 1,632,220 bytes  217,635.07 bytes/sec
total size is 1,631,722  speedup is 1.00


In [53]:
t = db.import_file(
    op.join(NOTEBOOK_NAME, NOTEBOOK_NAME + '.tsv'),
    tablename=NOTEBOOK_NAME,
    names=['uniprot_id', 'uniprot_mutation', 'ddg_exp', 'del_score_exp', 'del_class_exp'],
    sep='\t',
    na_values=['\\N'],
    keep_tmp=True,
)

In [54]:
t.name

'humsavar'

In [55]:
t.add_idx_column()

65452

In [56]:
t.create_indexes([
        (['uniprot_id', 'uniprot_mutation'], True),
    ])

In [57]:
t.compress()

File size before: 1.85 MB
File size after: 0.68 MB
File size savings: 0.68 MB (36.88 %)


# GIT LFS

In [61]:
!git lfs track ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*

Tracking ./mysqld/mutation_sets/humsavar.MYD
Tracking ./mysqld/mutation_sets/humsavar.MYI
Tracking ./mysqld/mutation_sets/humsavar.frm


In [63]:
!git add -f ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*